In [1]:
import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import pandas as pd

In [6]:
datos=pd.read_csv("dataset_dialogos/dataset_dialogos.csv")
datos=datos.loc[:len(datos)//10]["dialogue"]

In [15]:
datos=datos["dialogue"]
datos

In [8]:
datos_preprocesado=datos[50:100]["dialogue"]
datos_preprocesado

50    Enciérralo en el sótano.\n¡Mátelo! Es un monst...
51    ¡Déjalo ya!\nNos echarás encima a toda la coma...
52       Sal de aquí.\nTiene la fuerza de diez hombres.
53             Odiaba a Fritz.\nSiempre lo martirizaba.
54    Vamos, recupere la compostura.\n¿Qué podemos h...
55    ¿La tiene?\nSí, aquí está.\nEs muy fuerte. Med...
56                                         ¿Listo?\nSí.
57      ¿Le ha hecho daño?\nNo, estoy bien. No es nada.
58                                No.\nDemasiado tarde.
59    No quiero que lo vean.\nRápido, échenme una ma...
60           ¿Es ésta la puerta principal?\nSí, así es.
61              Está bien.\nNo me gusta, pero allá voy.
62    ¿lntenta provocar un incendio?\n¿Para qué es eso?
63    ¿Qué le pasa? Parece como si le hubiera cocead...
64    ¿Dónde estás, querida?\nVaya, conque estás ahí...
65    Déjelos solos.\nNo hay barandilla ni nada.\n¿C...
66                                   Henry.\nElizabeth.
67    ¡Henry!\n¡Victor! ¡Dr. Waldman!\nVengan en

In [9]:
#Prueba del tokenizer y embeddings model

In [10]:
from transformers import BertTokenizer,BertModel
model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
texto = """ "a ¿qué hora "pasa el, ciento siete.   """
tokens = tokenizer.tokenize(texto)
print(tokens)
# ['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
tokens_id=tokenizer.convert_tokens_to_ids(tokens)
#[169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
embedding_vector = model.embeddings.word_embeddings
embedding_vector 
#embedding_vector.shape

C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['"', 'a', '¿', 'qué', 'hora', '"', 'pasa', 'el', ',', 'cien', '##to', 'siete', '.']


Embedding(119547, 768, padding_idx=0)

In [12]:
#Funciones de etiquetado
from typing import List, Union
from transformers import BertTokenizer,BertModel

model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def separar_palabras(texto:str)->List[str]:
    primera_letra=0
    palabras=[]
    
    #palabras que terminan por espacio
    for i in range(len(texto)):
        if texto[i] == " ":
            palabra=texto[primera_letra:i]
            primera_letra=i+1
            palabras.append(palabra)
        #palabras que terminan por cambio de parrafo
        if texto[i]=="\n":
            palabra=texto[primera_letra:i]
            primera_letra=i+1  
            palabras.append(palabra)

        if texto[i]=="¡" or texto[i]=='"':
            primera_letra+=1
    
    #agrego ultima palabra
    palabras.append(texto[primera_letra:])
    return palabras

def clasificar_mayusculas(palabra:str)->int:
    """Toma una palabra y cuenta sus mayusculas"""
    cantidad=0
    for i in range(len(palabra)):
        if palabra[i].lower()!=palabra[i]:
            cantidad+=1
    res=0
    if cantidad==len(palabra): #todas son mayuscula
        res=3
    elif cantidad==1 and palabra[0].lower()!=palabra[0]: #solo la primera es mayuscula
        res=1
    elif cantidad>0: #alguna es mayuscula (iPhone tiene que estar aca)
        res=2
    else: #Ninguna de las anteriores
        res=0
    return res
    
def asignar_labels(datos: pd.Series, path=None)->pd.DataFrame:
    punt_inicial=[]
    instancias=[]
    token_ids=[]
    tokens=[]
    capitalizacion=[]
    pfinal=[]
    instancia=0
    print("Comenzando procesamiento")
    for dialogo in datos:
        if instancia % (len(datos)//10)==0:
            print(f"procesando un {instancia/len(datos)*100}%")
            
        palabras_originales= separar_palabras(dialogo)
        palabras_lower= [palabra.lower() for palabra in palabras_originales]
        instancia+=1
        #print(palabras_lower)

        punt_inicial_seguridad=[]
        instancias_seguridad=[]
        capitalizacion_seguridad=[]
        palabra_tokenizada_ids_seguridad=[]
        tokens_seguridad=[]
        pfinal_seguridad=[]
        es_seguro=True
        # hola, ¿Como estas. ? ### hola, ; ¿Como ; estas?
        #token  hola ,["¿", como] estas ?
        for idx, palabra in enumerate(palabras_lower):
            
            hay_signo_apertura=False 
            palabra_tokenizada= tokenizer.tokenize(palabra)

            #Recordartorio SON PALABRAS. ejemplos: '¿Que' o "escuela?" o "Franquito," o "Emi."  "¿ que ?"
            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[0]=="¿":
                    palabra_tokenizada=palabra_tokenizada[1:]
                    hay_signo_apertura=True
                    punt_inicial_seguridad.append(1) # ¿
                
             
            if len(palabra_tokenizada)>0:
                #estas ?
                if palabra_tokenizada[-1]=="?":        
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    
                    #relleno las demas listas
                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(1) #?
                    
                elif palabra_tokenizada[-1]==".":
                    
                    palabra_tokenizada=palabra_tokenizada[:-1] 

                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(2) #.
        
                elif palabra_tokenizada[-1]==",":
                    palabra_tokenizada=palabra_tokenizada[:-1]

                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(3) #,
        
                else:
                    pfinal_seguridad+=[0]*(len(palabra_tokenizada))
                    
            palabra_tokenizada_ids= tokenizer.convert_tokens_to_ids(palabra_tokenizada) 
            
            if hay_signo_apertura:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada)-1)
                
            else:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada))
            
            #capitalizacion
            clasificacion=None
            palabra_orginal= palabras_originales[idx]
            if len(palabra_tokenizada)>0:
                if palabra_orginal[0]=="¿" or palabra_orginal[0]=='"' or palabra_orginal[0]=='¡':
                    if palabra_orginal[-1]=='"' or palabra_orginal[-1]=='!' or palabra_orginal[-1]=='?' or palabra_orginal[-1]==',' or palabra_orginal[-1]=='.' or palabra_orginal[-1]=='?':
                        clasificacion= clasificar_mayusculas(palabra_orginal[1:-1])
                    else: 
                         clasificacion= clasificar_mayusculas(palabra_orginal[1:])
                else:   
                    clasificacion= clasificar_mayusculas(palabra_orginal)

            if len(palabra_tokenizada)==0:
                es_seguro=False 
                break
            
            capitalizacion_seguridad+= [clasificacion]*len(palabra_tokenizada)
            palabra_tokenizada_ids_seguridad+=palabra_tokenizada_ids
            tokens_seguridad+=palabra_tokenizada
            instancias_seguridad+=[instancia]*len(palabra_tokenizada)
        
        if es_seguro:
            capitalizacion+=capitalizacion_seguridad
            token_ids+= palabra_tokenizada_ids_seguridad
            tokens+= tokens_seguridad
            instancias+=instancias_seguridad
            punt_inicial+=punt_inicial_seguridad
            pfinal+=pfinal_seguridad
            
            if len(capitalizacion) != len(punt_inicial): #tamaño cantidad de token procesados
                print("dif inicial",instancia,dialogo)
                return
                
            if len(capitalizacion) != len(pfinal):
                print("dif final",instancia,dialogo)
                print(len(capitalizacion))
                print(len(punt_inicial))
                return
                
        else:
            instancia-=1
    
    
        if len(capitalizacion)!=len(punt_inicial):
            print("dif inicial",instancia)
            print(len(capitalizacion))
            print(len(punt_inicial))
            
        if len(capitalizacion)!=len(pfinal):
            print("dif final",instancia)
            print(len(capitalizacion))
            print(len(pfinal))
    
    datos_listos = pd.DataFrame({
    "instancia_id": pd.Series(instancias, dtype="int64"),   # o lo que corresponda
    "token": tokens,
    "token_id": pd.Series(token_ids, dtype="int64"),
    "punt_inicial": pd.Series(punt_inicial, dtype="int8"),
    "capitalizacion": pd.Series(capitalizacion, dtype="int8"),
    "pfinal": pd.Series(pfinal, dtype="int8")
})

    if path!=None:
        datos_listos.to_parquet(path, index=False)
    return datos_listos    
            

datos_listos=asignar_labels(datos_preprocesado,"datos_modelo.parquet")

C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Comenzando procesamiento
procesando un 0.0%
procesando un 10.0%
procesando un 20.0%
procesando un 30.0%
procesando un 40.0%
procesando un 50.0%
procesando un 60.0%
procesando un 70.0%
procesando un 80.0%
procesando un 90.0%


In [16]:
#primer decimo me entra en memoria tarda una hora
datos_listos=asignar_labels(datos,"datos_modelo.parquet") #subtitulso

Comenzando procesamiento
procesando un 0.0%
procesando un 9.999959966932687%
procesando un 19.999919933865375%
procesando un 29.99987990079806%
procesando un 39.99983986773075%
procesando un 49.999799834663435%
procesando un 59.99975980159612%
procesando un 69.9997197685288%
procesando un 79.9996797354615%
procesando un 89.99963970239418%


In [ ]:

#PARA DEJAR CORRIENDO TODA LA NOCHE

def asignar_labels_fraccional(datos: pd.Series,inicio:int = 0)->pd.DataFrame:
   
    max_intervalos=int(1/prop)
    
    for i in range(inicio,max_intervalos):
        chunck=datos[i*len(datos)*prop,i+1*len(datos)*prop]
        chunk_procesado=asignar_labels(chunk,f"datos_modelo_{i}.parquet")

    return chunck_procesado
        